In [1]:
import numpy as np
from PIL import Image

img = Image.open('photo.jpg')
I=np.asarray(img).copy()

In [2]:
I

array([[[77, 80, 85],
        [77, 80, 85],
        [77, 80, 85],
        ...,
        [67, 70, 75],
        [68, 71, 76],
        [66, 69, 74]],

       [[77, 80, 85],
        [77, 80, 85],
        [77, 80, 85],
        ...,
        [67, 70, 75],
        [66, 69, 74],
        [66, 69, 74]],

       [[77, 80, 85],
        [77, 80, 85],
        [77, 80, 85],
        ...,
        [66, 69, 74],
        [65, 68, 73],
        [63, 66, 71]],

       ...,

       [[71, 74, 79],
        [71, 74, 79],
        [71, 74, 79],
        ...,
        [66, 74, 77],
        [66, 74, 77],
        [71, 74, 79]],

       [[69, 72, 77],
        [70, 73, 78],
        [70, 73, 78],
        ...,
        [68, 73, 77],
        [68, 73, 77],
        [70, 73, 78]],

       [[70, 73, 78],
        [70, 73, 78],
        [72, 75, 80],
        ...,
        [68, 73, 77],
        [68, 73, 77],
        [68, 72, 75]]], dtype=uint8)

In [3]:
I2 = 255-I
img2 = Image.fromarray(I2)
img2.show()

In [8]:
R=I[:,:,0].astype('float')
G=I[:,:,1].astype('float')
B=I[:,:,2].astype('float')

In [9]:
from scipy import signal

In [10]:
data = ((R+G+B)/3).astype('uint8')

In [11]:
mask1=np.array([[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]])

In [12]:
data1=signal.convolve2d(data,mask1,boundary='symm',mode='same')
img1=Image.fromarray(data1)
img1.show()

In [13]:
mask2=np.array([[0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]])

In [14]:
data2=signal.convolve2d(data,mask2,boundary='symm',mode='same')
img2=Image.fromarray(data2)
img2.show()

# sobel 邊緣偵測(高頻)

In [15]:
Sx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
Sy = np.array([[-1,2,-1],[0,0,0],[1,2,1]])

In [16]:
Ix=signal.convolve2d(data,Sx,boundary='symm',mode='same')
Iy=signal.convolve2d(data,Sy,boundary='symm',mode='same')

In [17]:
data3 = (Ix**2)+(Iy**2)
data3_ID=np.sort(data3.reshape((-1,1))[:,0])
threshold = data3_ID[int(len(data3_ID)*0.526)]


In [18]:
(w,h)=img.size
data4 = np.zeros((h,w)).astype('uint8')
data4[data3<threshold]=255
img4 = Image.fromarray(data4)
img4.show()


|p|　|h11  h12  h13||x|<br>
|q| = |h21  h22  h23||y|<br>
|m|　|h31  h32  1 ||1|<br>

p= x * h11 + y * h12 + h13<br>
m= x * h31 + y * h32 + 1 <br>
q= x * h21 + y * h22 + h23

p'=p/m<br>
q'=q/m<br>

p'(x * h31 + y * h32 + 1) = x * h11 + y * h12 + h13<br>
p'= x * h11 + y * h12 + h13 -  p'* x * h31 + p'* y * h32<br>
q'(x * h31 + y * h32 + 1) = x * h21 + y * h22 + h23<br>
q'= x * h21 + y * h22 + h23 -  q'* x * h31 + q'* y * h32

|x,y,1,0,0,0,-p'x,-p'y||h11|<br>
|　　　　　　　　||h12|<br>
|　　　　　　　　||h13|<br>
|　　　　　　　　||h21|<br>
|　　　　　　　　||h22|=8|p'|<br>
|　　　　　　　　||h23|　|q'|<br>
|　　　　　　　　||h31|<br>
|　　　　　　　　||h32|